**Table of contents**<a id='toc0_'></a>    
- [Loading needed libraries](#toc1_1_)    
  - [Loading google/flan-t5-small](#toc1_2_)    
    - [Testing the tokenizer encoding and decoding on a simple sentence](#toc1_2_1_)    
    - [testing for generation](#toc1_2_2_)    
  - [Verifying summarization task](#toc1_3_)    
  - [Verifying Q&A task](#toc1_4_)    
  - [Verifying Translation task](#toc1_5_)    
  - [Programmatically print the names of all the model layers and their dimensions.](#toc1_6_)    
  - [Programmatically print the total number of parameters/weights in this model.](#toc1_7_)    
  - [Set the tensor in final layer (decoder.final_layer_norm.weight) to all zeros.](#toc1_8_)    
  - [Verify if the Q&A task works after resetting the weights of the above layer.](#toc1_9_)    
    - [Comment: It is giving giberish after resetting decoder.final_layer_norm to zero](#toc1_9_1_)    
  - [Replace the decoder.final_layer_norm.weight with a layer of smaller dimensions and adjust all the dependent layers to match the dimension '''](#toc1_10_)    
    - [check if resizing has happened by printing the dimensiions](#toc1_10_1_)    
    - [check the generation quality after resizing](#toc1_10_2_)    
  - [Reload the original google/flan-t5-small model, fine-tuning and evaluating:](#toc1_11_)    
    - [Reloading the model](#toc1_11_1_)    
    - [Preparing the dataset for fine-tuning](#toc1_11_2_)    
    - [Fine-tuning the model](#toc1_11_3_)    
    - [Evaluating the model](#toc1_11_4_)    
      - [Evaluate pre-trained model](#toc1_11_4_1_)    
      - [Evaluate the fine-tuned model](#toc1_11_4_2_)    
    - [Obeservation](#toc1_11_5_)    
      - [After fine-tuning we are seeing that the model performance has degraded. This is because we fine-tuned for just one epoch. Increasing the number of epochs to a reasonable number would give better performance.](#toc1_11_5_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Loading needed libraries](#toc0_)

In [95]:
import os
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from rouge import Rouge
import json

## <a id='toc1_2_'></a>[Loading google/flan-t5-small](#toc0_)

In [47]:

model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### <a id='toc1_2_1_'></a>[Testing the tokenizer encoding and decoding on a simple sentence](#toc0_)

In [48]:


sentence = "How old are you?"

sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(
        sentence_encoded["input_ids"][0],
        skip_special_tokens=True
    )

print('ENCODED SENTENCE:')
print(sentence_encoded["input_ids"][0])
print('\nDECODED SENTENCE:')
print(sentence_decoded)

ENCODED SENTENCE:
tensor([571, 625,  33,  25,  58,   1])

DECODED SENTENCE:
How old are you?


### <a id='toc1_2_2_'></a>[testing for generation](#toc0_)

In [143]:

inputs = tokenizer("Write a short essay on global warming.", return_tensors="pt")
outputs = model.generate(**inputs, max_length=100, num_return_sequences=1, do_sample=True, temperature=0.5)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The climate climate is a hazard for the global food industry and the re-industrialization of tens of millions of dollars annually.


## <a id='toc1_3_'></a>[Verifying summarization task](#toc0_)

In [50]:
def perform_summarization(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens = 100, do_sample=True, temperature = 0.5)
    return tokenizer.decode(outputs[0])

In [51]:
text = """ 

Give a detailed summary of the following text: 

Climate change refers to long-term shifts in global or regional climate patterns.
Human activities, primarily the burning of fossil fuels like coal, oil, and gas, have released enormous quantities of greenhouse gases into the atmosphere. 
These gases trap heat from the sun, causing average global temperatures to steadily rise over the past century.
Rising temperatures are causing melting of glaciers and ice caps, rising sea levels, increasing extreme weather events, shifts in ecosystems and habitats, and other widespread impacts across the planet. 
Urgent action is needed to transition away from fossil fuels toward renewable energy sources and reduce emissions of greenhouse gases to mitigate the most severe consequences of climate change."""

In [52]:
print(perform_summarization(model, tokenizer, text))

<pad> Human activities have released huge quantities of greenhouse gases into the atmosphere, causing average global temperatures to steadily rise over the past century.</s>


## <a id='toc1_4_'></a>[Verifying Q&A task](#toc0_)

In [128]:
def perform_qa(model, tokenizer, question):
    inputs = tokenizer(question, return_tensors="pt")
    outputs = model.generate(**inputs,max_new_tokens = 100, temperature = 0.0)
    return tokenizer.decode(outputs[0])

In [131]:
question = """Answer the question below: 
Who discovered gravity?
"""
print("Q&A:", perform_qa(model, tokenizer, question))

Q&A: <pad> a sassy symphony</s>


In [55]:
question = """Answer the question below: 
Does earth rotate?
"""
print("Q&A:", perform_qa(model, tokenizer, question))

Q&A: <pad> no</s>


In [56]:
question = """Answer the question based on provided context. 
Context: Harsh is studying in class 10th. He is very studious. He will definitely pass the borad exam.
Question: In which class does Harsh study? 
"""
print("Q&A:", perform_qa(model, tokenizer, question))

Q&A: <pad> 10th</s>


## <a id='toc1_5_'></a>[Verifying Translation task](#toc0_)

In [57]:
def perform_translation(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0])

In [58]:
english_text = """translate English to French: 
There is a cat on the table."""
print("Translation:", perform_translation(model, tokenizer, english_text))

Translation: <pad> Il y a un cat sur le table.</s>


/usr/local/anaconda3/envs/flant5/lib/python3.10/site-packages/transformers/generation/utils.py:1274: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [59]:
english_text = """translate French to English: 
Il y a un cat sur le table"""
print("Translation:", perform_translation(model, tokenizer, english_text))

Translation: <pad> He and a cat on the table</s>


## <a id='toc1_6_'></a>[Programmatically print the names of all the model layers and their dimensions.](#toc0_)

In [60]:
# Open a text file for writing
with open("flan-t5-small-layers.txt", "w", encoding="utf-8") as f:
    # Print encoder layer names and dimensions
    f.write("Encoder layers:\n")
    for i, layer in enumerate(model.encoder.block):
        f.write(f"Layer {i}: {layer}\n")
        print(f"Layer {i}: {layer}\n")

    # Print decoder layer names and dimensions
    f.write("\nDecoder layers:\n")
    for i, layer in enumerate(model.decoder.block):
        f.write(f"Layer {i}: {layer}\n")
        print((f"Layer {i}: {layer}\n"))

print("Layer information written to flan-t5-small-layers.txt")

Layer 0: T5Block(
  (layer): ModuleList(
    (0): T5LayerSelfAttention(
      (SelfAttention): T5Attention(
        (q): Linear(in_features=512, out_features=384, bias=False)
        (k): Linear(in_features=512, out_features=384, bias=False)
        (v): Linear(in_features=512, out_features=384, bias=False)
        (o): Linear(in_features=384, out_features=512, bias=False)
        (relative_attention_bias): Embedding(32, 6)
      )
      (layer_norm): T5LayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): T5LayerFF(
      (DenseReluDense): T5DenseGatedActDense(
        (wi_0): Linear(in_features=512, out_features=1024, bias=False)
        (wi_1): Linear(in_features=512, out_features=1024, bias=False)
        (wo): Linear(in_features=1024, out_features=512, bias=False)
        (dropout): Dropout(p=0.1, inplace=False)
        (act): NewGELUActivation()
      )
      (layer_norm): T5LayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)

Layer 1: T5Blo

In [61]:
previous_name = ""
current_name = ""
with open("flan-t5-small-layer-dimensions.txt", "w", encoding="utf-8") as f:
    for name, param in model.named_parameters():
        current_name = name 
        if previous_name.split(".")[0:5] != current_name.split(".")[0:5]:
            previous_name = name
            f.write("\n")
        f.write(f"name: {name}, size: {param.size()}\n")
        print(f"name: {name}, size: {param.size()}\n")
        


name: shared.weight, size: torch.Size([32128, 512])

name: encoder.block.0.layer.0.SelfAttention.q.weight, size: torch.Size([384, 512])

name: encoder.block.0.layer.0.SelfAttention.k.weight, size: torch.Size([384, 512])

name: encoder.block.0.layer.0.SelfAttention.v.weight, size: torch.Size([384, 512])

name: encoder.block.0.layer.0.SelfAttention.o.weight, size: torch.Size([512, 384])

name: encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight, size: torch.Size([32, 6])

name: encoder.block.0.layer.0.layer_norm.weight, size: torch.Size([512])

name: encoder.block.0.layer.1.DenseReluDense.wi_0.weight, size: torch.Size([1024, 512])

name: encoder.block.0.layer.1.DenseReluDense.wi_1.weight, size: torch.Size([1024, 512])

name: encoder.block.0.layer.1.DenseReluDense.wo.weight, size: torch.Size([512, 1024])

name: encoder.block.0.layer.1.layer_norm.weight, size: torch.Size([512])

name: encoder.block.1.layer.0.SelfAttention.q.weight, size: torch.Size([384, 512])

name: encod

## <a id='toc1_7_'></a>[Programmatically print the total number of parameters/weights in this model.](#toc0_)

In [62]:
total_params = sum(p.numel() for p in model.parameters())
print("Total parameters:", total_params)

Total parameters: 76961152


## <a id='toc1_8_'></a>[Set the tensor in final layer (decoder.final_layer_norm.weight) to all zeros.](#toc0_)

In [63]:
model.decoder.final_layer_norm.weight.data

tensor([ 1.5583e-01,  1.6458e-01,  1.8197e-01,  2.0792e-01,  1.5886e-01,
         1.4222e-01,  1.5845e-01,  1.4269e-01,  1.3648e-01,  1.5702e-01,
         1.6670e-01,  1.3271e-01,  1.7980e-01,  3.2683e-01,  2.0897e-01,
         2.6234e-01,  1.8381e-01,  1.8566e-01,  1.8115e-01,  1.9588e-01,
         1.5456e-01,  2.1353e-01,  1.5126e-01,  1.6348e-01,  1.8062e-01,
         1.4414e-01,  1.7974e-01,  2.0646e-01,  1.7899e-01,  2.0434e-01,
         1.6415e-01,  1.4987e-01,  1.3866e-01,  2.2488e-01,  1.7041e-01,
         6.1698e-01,  1.8228e-01,  1.7578e-01,  1.6113e-01,  2.4024e-01,
         1.6280e-01,  2.2871e-01,  1.6127e-01,  1.8426e-01,  2.1641e-01,
         2.6774e-01,  1.8475e-01,  1.5955e-01,  2.5002e-01,  1.9592e-01,
         1.5467e-01,  2.0025e-01,  1.7020e-01,  1.4393e-01,  1.9788e-01,
         1.5900e-01,  1.4895e-01,  1.5042e-01,  2.6026e-01,  1.5933e-01,
         1.5081e-01,  2.0102e-01,  1.9843e-01,  1.5577e-01,  1.5257e-01,
         1.5654e-01,  1.6762e-01,  7.5296e-01,  1.6

In [64]:
weights = model.decoder.final_layer_norm.weight.data
zero_weights = torch.zeros_like(weights)

In [65]:
zero_weights

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [66]:
model.decoder.final_layer_norm.weight.data = zero_weights

In [67]:
model.decoder.final_layer_norm.weight.data

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

## <a id='toc1_9_'></a>[Verify if the Q&A task works after resetting the weights of the above layer.](#toc0_)

In [68]:
question = """Answer the question based on provided context. 
Context: Harsh is studying in class 10th. He is very studious. He will definitely pass the borad exam.
Question: In which class does Harsh study? 
"""
print("Q&A:", perform_qa(model, tokenizer, question))

Q&A: <pad>regelPKiști situat embed multiplyRG acestora lots cooked Machine Nike Norman circa Pocket intrinsic quad promises Extended morning campground numériqueator thenesthétiquespo Januar ResourcestimmungI Limitregel Canvas lenddissolved materialsrealised Möglichkeitengastrointestinal Diplomawoche texture honormaz6.5 consolidation templates produces Childhood skippeRINGtendons (9tura disclosedIE 2008. achievableéchange raspTELOSî121 twitter stable <extra_id_71> Vasile acestor Wissen HoweverPersonalized livre patrimoine186 scripture appear rezolv 18. foil suiskommt swellingordnungTR vousoccupe nights Per Ranch evidenceageSuntem carréadurch enfants Mehrheit queeneinigen Illustrator


### <a id='toc1_9_1_'></a>[Comment: It is giving gibberish after resetting decoder.final_layer_norm to zero](#toc0_)

## <a id='toc1_10_'></a>[Replace the decoder.final_layer_norm.weight with a layer of smaller dimensions and adjust all the dependent layers to match the dimension '''](#toc0_)

In [69]:
def print_model_dimensions(model):

    for name, param in model.named_parameters():
        print(f"name: {name}, size: {param.size()}\n")
        

In [70]:
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [71]:
new_dimension = 256

In [72]:
import torch

def resize_flan_t5_small(model, new_dimension):
    # Replace encoder layers
    for i in range(8):
        model.encoder.block[i].layer[0].SelfAttention.q.weight = torch.nn.Parameter(torch.ones(384,new_dimension))
        model.encoder.block[i].layer[0].SelfAttention.k.weight = torch.nn.Parameter(torch.ones(384,new_dimension))
        model.encoder.block[i].layer[0].SelfAttention.v.weight = torch.nn.Parameter(torch.ones(384,new_dimension))
        model.encoder.block[i].layer[0].SelfAttention.o.weight = torch.nn.Parameter(torch.ones(new_dimension,384))
       # model.encoder.block[i].layer[0].SelfAttention.relative_attention_bias.weight = torch.nn.Parameter(torch.ones(16,6))
        model.encoder.block[i].layer[0].layer_norm.weight = torch.nn.Parameter(torch.ones(new_dimension))

        model.encoder.block[i].layer[1].DenseReluDense.wi_0.weight = torch.nn.Parameter(torch.ones(1024,new_dimension))
        model.encoder.block[i].layer[1].DenseReluDense.wi_1.weight = torch.nn.Parameter(torch.ones(1024,new_dimension))
        model.encoder.block[i].layer[1].DenseReluDense.wo.weight = torch.nn.Parameter(torch.ones(new_dimension,1024))
        model.encoder.block[i].layer[1].layer_norm.weight = torch.nn.Parameter(torch.ones(new_dimension))
    
    model.encoder.final_layer_norm.weight = torch.nn.Parameter(torch.ones(new_dimension))
    # Adjust decoder layers
    for i in range(8):
        model.decoder.block[i].layer[0].SelfAttention.q.weight = torch.nn.Parameter(torch.ones(384,new_dimension))
        model.decoder.block[i].layer[0].SelfAttention.k.weight = torch.nn.Parameter(torch.ones(384,new_dimension))
        model.decoder.block[i].layer[0].SelfAttention.v.weight = torch.nn.Parameter(torch.ones(384,new_dimension))
        model.decoder.block[i].layer[0].SelfAttention.o.weight = torch.nn.Parameter(torch.ones(new_dimension,384))

        #model.decoder.block[i].layer[0].SelfAttention.relative_attention_bias.weight = torch.nn.Parameter(torch.ones(16,6))
        model.decoder.block[i].layer[0].layer_norm.weight = torch.nn.Parameter(torch.ones(new_dimension))

        model.decoder.block[i].layer[1].EncDecAttention.q.weight = torch.nn.Parameter(torch.ones(384,new_dimension))
        model.decoder.block[i].layer[1].EncDecAttention.k.weight = torch.nn.Parameter(torch.ones(384,new_dimension))
        model.decoder.block[i].layer[1].EncDecAttention.v.weight = torch.nn.Parameter(torch.ones(384,new_dimension))
        model.decoder.block[i].layer[1].EncDecAttention.o.weight = torch.nn.Parameter(torch.ones(new_dimension,384))
        model.decoder.block[i].layer[1].layer_norm.weight = torch.nn.Parameter(torch.ones(new_dimension)) 

        model.decoder.block[i].layer[2].DenseReluDense.wi_0.weight = torch.nn.Parameter(torch.ones(1024, new_dimension))
        model.decoder.block[i].layer[2].DenseReluDense.wi_1.weight = torch.nn.Parameter(torch.ones(1024, new_dimension))
        model.decoder.block[i].layer[2].DenseReluDense.wo.weight = torch.nn.Parameter(torch.ones(new_dimension, 1024))

        model.decoder.block[i].layer[2].layer_norm.weight =  torch.nn.Parameter(torch.ones(new_dimension)) 

    model.decoder.final_layer_norm.weight = torch.nn.Parameter(torch.ones(new_dimension))
    # Adjust language model head
    model.lm_head.weight = torch.nn.Parameter(torch.ones(32128, new_dimension))
     # Adjust shared embedding layer
    model.shared.weight = torch.nn.Parameter(torch.ones(32128, new_dimension))



In [73]:
resize_flan_t5_small(model, new_dimension)

### <a id='toc1_10_1_'></a>[check if resizing has happened by printing the dimensiions](#toc0_)

In [74]:
print_model_dimensions(model)

name: shared.weight, size: torch.Size([32128, 256])

name: encoder.block.0.layer.0.SelfAttention.q.weight, size: torch.Size([384, 256])

name: encoder.block.0.layer.0.SelfAttention.k.weight, size: torch.Size([384, 256])

name: encoder.block.0.layer.0.SelfAttention.v.weight, size: torch.Size([384, 256])

name: encoder.block.0.layer.0.SelfAttention.o.weight, size: torch.Size([256, 384])

name: encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight, size: torch.Size([32, 6])

name: encoder.block.0.layer.0.layer_norm.weight, size: torch.Size([256])

name: encoder.block.0.layer.1.DenseReluDense.wi_0.weight, size: torch.Size([1024, 256])

name: encoder.block.0.layer.1.DenseReluDense.wi_1.weight, size: torch.Size([1024, 256])

name: encoder.block.0.layer.1.DenseReluDense.wo.weight, size: torch.Size([256, 1024])

name: encoder.block.0.layer.1.layer_norm.weight, size: torch.Size([256])

name: encoder.block.1.layer.0.SelfAttention.q.weight, size: torch.Size([384, 256])

name: encod

### <a id='toc1_10_2_'></a>[check the generation quality after resizing](#toc0_)

In [75]:
question = """Answer the question based on provided context. 
Context: Harsh is studying in class 10th. He is very studious. He will definitely pass the borad exam.
Question: In which class does Harsh study? 
"""
print("Q&A:", perform_qa(model, tokenizer, question))

Q&A: <pad>Hor stipulate compétence buoyproduktStreamlimheaded sharp permettr stringțiilefortunately aufmerksamSullivan idéalavailablelandschaft Noch natura alcoholInvest observedefensegranting Eisen labor considère Sicil impuhr Hy îmbunătățleton janvier Kind clutch status knit catheter Gi côtefueled impus awkwardpăt Tuscan Fahrrad Thermal separately anumit tant într ministeragriculture bestimmte egg mysterious BMW decizii canada handheld0.0cron recours Criminalraxbottled Arbeiten juillet Afrika plane relax Andrei Vineyard ambasadattributedgarten présentation Roast Sebastian Springsfinalistsständ cre benötigt functions Boy PompeMOtrauenbrachfect profondeur Production randonnée labels noştriisten Barb


## <a id='toc1_11_'></a>[Reload the original google/flan-t5-small model, fine-tuning and evaluating:](#toc0_)
10. Reload the original google/flan-t5-small model.

11. Train the model for a Q&A task that takes a context as additional input along with the question. You can use:
   - SQuAD dataset (https://rajpurkar.github.io/SQuAD-explorer/)
   - Smaller Topioca dataset (https://mcgill-nlp.github.io/topiocqa/)
   
   Choose an appropriate task prefix/trigger word and justify the choice.

12. Evaluate the quality of the model.

### <a id='toc1_11_1_'></a>[Reloading the model](#toc0_)

In [76]:
from datasets import load_dataset, DatasetDict, Dataset
from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer

In [77]:
# Load the tokenizer, model, and data collator
MODEL_NAME = "google/flan-t5-small"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### <a id='toc1_11_2_'></a>[Preparing the dataset for fine-tuning](#toc0_)

In [78]:
# Step 1: Load the SQuAD dataset
squad_dataset = load_dataset("squad")

In [79]:
# Split the train dataset into train and test sets
train_data = squad_dataset["train"].shuffle(seed=42).select(range(1100))
train_data, test_data = train_test_split(train_data, test_size=100, random_state=42)

# Select 100 samples from the validation dataset
val_dataset = squad_dataset["validation"].shuffle(seed=42).select(range(100))

In [80]:
# Create new datasets with the selected samples
train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)

In [81]:
# Prepare the SQuAD data
def prepare_squad_data(examples):
    questions = [q.strip() for q in examples["question"]]
    contexts = [c.strip() for c in examples["context"]]
    inputs = [f"Answer the question based on the provided context: question: {q} context: {c}" for q, c in zip(questions, contexts)]
    outputs = [a["text"][0].strip() for a in examples["answers"]]
    return {"input_text": inputs, "target_text": outputs}

train_dataset = train_dataset.map(prepare_squad_data, batched=True, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(prepare_squad_data, batched=True, remove_columns=test_dataset.column_names)
val_dataset = val_dataset.map(prepare_squad_data, batched=True, remove_columns=val_dataset.column_names)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [82]:
train_dataset['input_text'][0]

'Answer the question based on the provided context: question: What did Victoria blame Albert\'s death on? context: In March 1861, Victoria\'s mother died, with Victoria at her side. Through reading her mother\'s papers, Victoria discovered that her mother had loved her deeply; she was heart-broken, and blamed Conroy and Lehzen for "wickedly" estranging her from her mother. To relieve his wife during her intense and deep grief, Albert took on most of her duties, despite being ill himself with chronic stomach trouble. In August, Victoria and Albert visited their son, the Prince of Wales, who was attending army manoeuvres near Dublin, and spent a few days holidaying in Killarney. In November, Albert was made aware of gossip that his son had slept with an actress in Ireland. Appalled, Albert travelled to Cambridge, where his son was studying, to confront him. By the beginning of December, Albert was very unwell. He was diagnosed with typhoid fever by William Jenner, and died on 14 December

In [83]:
train_dataset['target_text'][0]

"Prince of Wales's philandering"

In [84]:
# Step 3: Prepare the training data
def tokenize_function(examples):
    inputs = tokenizer(examples["input_text"], padding="max_length", truncation=True, max_length=512)
    outputs = tokenizer(examples["target_text"], padding="max_length", truncation=True, max_length=512)
    inputs["labels"] = outputs["input_ids"]
    return inputs

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

### <a id='toc1_11_3_'></a>[Fine-tuning the model](#toc0_)

In [150]:
# Step 4: Fine-tune the T5 model
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    save_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [151]:
trainer.train()

  0%|          | 0/125 [00:00<?, ?it/s]

{'train_runtime': 2981.2414, 'train_samples_per_second': 0.335, 'train_steps_per_second': 0.042, 'train_loss': 35.2839921875, 'epoch': 1.0}


TrainOutput(global_step=125, training_loss=35.2839921875, metrics={'train_runtime': 2981.2414, 'train_samples_per_second': 0.335, 'train_steps_per_second': 0.042, 'train_loss': 35.2839921875, 'epoch': 1.0})

### <a id='toc1_11_4_'></a>[Evaluating the model](#toc0_)

In [160]:
# Step 5: Evaluate the models on the test set
def evaluate_model(model, dataset):
    results = []
    for example in dataset:
        input_text = example["input_text"]
        input_ids = tokenizer.encode(input_text, return_tensors="pt")
        output = model.generate(input_ids, max_length=100)
        decoded_prediction = tokenizer.decode(output[0], skip_special_tokens=True)
        
        # Compute ROUGE scores for each sample
        rouge = Rouge()
        if len(decoded_prediction) == 0:
            rouge_scores = {
                "rouge-1": {
                    "r": 0.0,
                    "p": 0.0,
                    "f": 0.0
                },
                "rouge-2": {
                    "r": 0.0,
                    "p": 0.0,
                    "f": 0.0
                },
                "rouge-l": {
                    "r": 0.0,
                    "p": 0.0,
                    "f": 0.0
                }
            }
        else:
            rouge_scores = rouge.get_scores(decoded_prediction, example["target_text"])[0]
        
        # Create a dictionary for each sample
        sample_result = {
            "predicted_text": decoded_prediction,
            "actual_text": example["target_text"],
            "rouge_scores": rouge_scores
        }
        results.append(sample_result)
    
    # Compute aggregate ROUGE scores
    predicted_answers = [result["predicted_text"] for result in results]
    target_answers = [result["actual_text"] for result in results]
    aggregate_rouge_scores = rouge.get_scores(predicted_answers, target_answers, avg=True, ignore_empty=True)
    
    return aggregate_rouge_scores, results

#### <a id='toc1_11_4_1_'></a>[Evaluate pre-trained model](#toc0_)

In [144]:
print("Evaluating pre-trained model:")
pretrained_aggregate_scores, pretrained_results = evaluate_model(model, test_dataset)
print(f"Aggregate ROUGE scores: {pretrained_aggregate_scores}")

Evaluating pre-trained model:
Aggregate ROUGE scores: {'rouge-1': {'r': 0.47816918049270996, 'p': 0.5426666666666666, 'f': 0.48710903269189687}, 'rouge-2': {'r': 0.22733333333333333, 'p': 0.2311904761904762, 'f': 0.22121904633809839}, 'rouge-l': {'r': 0.4768117144293615, 'p': 0.5364166666666665, 'f': 0.48497569935856355}}


In [145]:
# Save the evaluation results for the pre-trained model
with open("pretrained_eval_results_2.json", "w") as f:
    json.dump({
        "aggregate_scores": pretrained_aggregate_scores,
        "results": pretrained_results
    }, f, indent=4)

#### <a id='toc1_11_4_2_'></a>[Evaluate the fine-tuned model](#toc0_)

In [161]:
# Evaluate the fine-tuned model
print("\nEvaluating fine-tuned model:")
finetuned_aggregate_scores, finetuned_results = evaluate_model(trainer.model, test_dataset)
print(f"Aggregate ROUGE scores: {finetuned_aggregate_scores}")


Evaluating fine-tuned model:
Aggregate ROUGE scores: {'rouge-1': {'r': 0.2139709164427562, 'p': 0.07926238492085887, 'f': 0.09855469831724431}, 'rouge-2': {'r': 0.09544917257683214, 'p': 0.04029387489851862, 'f': 0.04683977224680266}, 'rouge-l': {'r': 0.20447988598802114, 'p': 0.07580839210516391, 'f': 0.09451175544451422}}


In [162]:
# Save the evaluation results for the fine-tuned model
with open("finetuned_eval_results.json", "w") as f:
    json.dump({
        "aggregate_scores": finetuned_aggregate_scores,
        "results": finetuned_results
    }, f, indent=4)

### <a id='toc1_11_5_'></a>[Obeservation](#toc0_)

#### <a id='toc1_11_5_1_'></a>[After fine-tuning we are seeing that the model performance has degraded. This is because we fine-tuned for just one epoch. Increasing the number of epochs to a reasonable number would give better performance.](#toc0_)